# SEGAN_OM

a GAN based filter method for speech enhancement

### 1) Data pipeline

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import numpy as np
import scipy.io.wavfile as wavfile
from io import *
import os.path

In [2]:
SAMPLE_SIZE=2*15
kernel_size=31

In [22]:
def load_file(file_path, window = 2**14, stride=0.5, sampling=16000):
    """
    Loads a wav file and returns it as a Tensor.
    Inputs:
    file_path: the path of the wav file
    window: optional, size of each sample batch
    stride: optional, defult = 0.5
    sampling: optional, sampling rate, default 16000
    Returns:
    slices: a list of TF-Tensors containing the values of the wav after cropping the last part exceding the windows width
    """
    sampling_rate = tf.constant(sampling,dtype=tf.int32,shape=())
    slices = []
    window = int(window*stride)
    file = tf.io.read_file(file_path)
    input_sequence = tf.audio.decode_wav(file)
    if not tf.math.equal(sampling_rate,input_sequence[1]):
        raise ValueError(f'Sampling rate is expected to be {sampling}! Got {input_sequence[1]}')
    input_sequence_length = input_sequence[0].shape[0]
    for win_start in range(0,input_sequence_length,window):
        slice_window = input_sequence[0][win_start:win_start+window]
        if slice_window.shape[0] == window:
            slices.append(slice_window)
    return np.array(slices, dtype=np.float64)

In [23]:
# From TFRecord Tutorial
# The following functions can be used to convert a value to a type compatible

def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [24]:
def make_example():
    """
    Inputs:
    Returns:
    """
    return 0

In [25]:
def create_record():
    """
    Inputs:
    Returns:
    """
    return 0

In [26]:
path = "./Dataset/clean"
out_filepath = './Dataset/save10.tfrecords'
out_file = tf.io.TFRecordWriter(out_filepath)
signals_clean = []
for file in  os.listdir(path):
    sequence=load_file(os.path.join(path,file))
    signals_clean.append(sequence)
    break

In [29]:
for wav in signals_clean:
    wav_raw = wav.tostring()
    example = tf.train.Example(features=tf.train.Features(feature={
        'wav_raw': _bytes_feature(wav_raw)}))